In [3]:
# Install packages
!pip install nltk
!pip install numpy
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
# !pip install google-cloud-storage
!pip install pandas
# !pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b7182c9f306421092de6248b4ffcec703befb7f14086362d022c6920a5eee500
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 2.2 MB/s eta 0:00:00


In [4]:
# Imports
import evaluate
import numpy as np
import pandas as pd
import rouge_score
import sacrebleu
from sacrebleu import CHRF
# import bert_score
import ast

In [5]:
# Metrics
rouge_score = evaluate.load("rouge")
bleu_score = evaluate.load("bleu")
sacrebleu_score = evaluate.load("sacrebleu")
chrf = evaluate.load("chrf")
# bertscore = evaluate.load("bertscore")

In [6]:
def split_verbalisation(text):
  split_text = text.split("Verbalisation: ") # For non-eng blueprints
  # split_text = text.split("): ")  # For eng blueprints

  if len(split_text) > 1:
    return split_text[1]
  else:
    return text

In [7]:
# Load dataset
df = pd.read_csv("mt5-small-blueprints-pt-preds.csv", sep='\t')

df['preds'] = df['preds'].apply(split_verbalisation)

# df2 = pd.read_csv("eng_test.csv", sep='\t')
# df2 = pd.read_csv("test_full.csv", sep='\t')
df2 = pd.read_csv("pt_test.csv", sep='\t')
df2['table_text'] = df2['table_text'].apply(ast.literal_eval)

# Remove Blueprint: repetitions
# mask_a = df['preds'].str.contains('Blueprint:')
# df = df[~mask_a]
# df2 = df2[~mask_a]

preds = df['preds'].tolist()
# refs = df['refs'].tolist()

refs = df2['table_text'].tolist()

In [ ]:
len(df2)

100

In [8]:
# Find the length of the longest list
max_length = max(len(sublist) for sublist in refs)

# Pad each sublist to match the length of the longest list
for sublist in refs:
    while len(sublist) < max_length:
        sublist.append("")

In [ ]:
df.iloc[3]['refs']

In [ ]:
df2.iloc[3]['table_text']

In [9]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_path = 'adenhaus/mt5-large-stata'
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)

class RegressionLogitsProcessor(torch.nn.Module):
    def __init__(self, extra_token_id):
        super().__init__()
        self.extra_token_id = extra_token_id

    def __call__(self, input_ids, scores):
        extra_token_logit = scores[:, :, self.extra_token_id]
        return extra_token_logit

def preprocess_inference_input(input_text):
    input_encoded = tokenizer(input_text, return_tensors='pt')
    return input_encoded

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
unused_token = "<extra_id_1>"

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def do_regression(input_str):
  input_data = preprocess_inference_input(input_str).to(device)

  logits_processor = RegressionLogitsProcessor(tokenizer.get_vocab()[unused_token])

  output_sequences = model.generate(
      **input_data,
      max_length=2,  # Generate just the regression token
      do_sample=False,  # Important: Disable sampling for deterministic output
      return_dict_in_generate=True,  # Get the scores directly
      output_scores=True
  )

  # Extract the logit
  unused_token_id = tokenizer.get_vocab()[unused_token]
  regression_logit = output_sequences.scores[0][0][unused_token_id]

  regression_score = sigmoid(regression_logit).item()

  return regression_score

In [11]:
import pandas as pd

# df = pd.read_csv('test_stata.csv')

predictions = []

# Create strings and call predict function for each row
for index, row in df.iterrows():
    input_string = row['linearized_input'] + ' [output] ' + row['preds']

    prediction = do_regression(input_string)
    print(row['linearized_input'])
    print(row['preds'])
    print(prediction)
    print()
    predictions.append(prediction)

# Display the results
print('STATA-QE:')
print(sum(predictions) / len(predictions))
print()

Média de idades na primeira relação sexual, no primeiro casamento e no nascimento do primeiro filho nos homens com idades entre os 30 e os 34 anos, por residência | Idade (anos) | (Primeira relação sexual, Urbana, 18.4) (Primeiro casamento, Urbana, 25.6) (Nascimento do primeiro filho, Urbana, 26.9) (Primeira relação sexual, Rural, 18.5) (Primeiro casamento, Rural, 24.1) (Nascimento do primeiro filho, Rural, 25.1) (Primeira relação sexual, Urbana, 21.3) (Primeiro casamento, Urbana, 27.7) (Nascimento do primeiro filho, Urbana, 28.2) (Primeira relação sexual, Rural, 20.8) (Primeiro casamento, Rural, 25) (Nascimento do primeiro filho, Rural, 25.6) (Primeira relação sexual, Urbana, 21.5) (Primeiro casamento, Urbana, 27.4) (Nascimento do primeiro filho, Urbana, 29.2) (Primeira relação sexual, Rural, 22.1) (Primeiro casamento, Rural, 25.5) (Nascimento do primeiro filho, Rural, 26.8) (Primeira relação sexual, Urbana, 21.7) (Primeiro casamento, Urbana, 27) (Nascimento do primeiro filho, Urbana,

In [12]:
# BLEU
score = bleu_score.compute(
    predictions=preds,
    references=refs
)
print("BLEU: " + str(score))
print()

BLEU: {'bleu': 0.16019184084015867, 'precisions': [0.4970131421744325, 0.20264681555004135, 0.1113978494623656, 0.05869175627240143], 'brevity_penalty': 1.0, 'length_ratio': 193.15384615384616, 'translation_length': 2511, 'reference_length': 13}



In [13]:
# CHRF
score = chrf.compute(
    predictions=preds,
    references=refs,
    lowercase=True
)
print("CHRF: " + str(score))
print()

CHRF: {'score': 34.469256192200206, 'char_order': 6, 'word_order': 0, 'beta': 2}



In [ ]:
# ROUGE
scores = rouge_score.compute(
    predictions=preds,
    references=refs,
    rouge_types=["rouge1", "rouge2", "rougeL"]
)

print("ROUGE:")
print(scores)
print()

In [ ]:
# Test
test_refs = [['Only 18% of women own a house, either alone or jointly, and only 15% own land.', 'In comparison, men are more than twice as likely to own a home alone or jointly (40%).', 'Men are also more than twice as likely to own land alone or jointly (34%).']]
test_preds = ['15% of women between the ages of 15 and 49 own land alone or jointly.']

score = chrf.compute(
    predictions=test_preds,
    references=test_refs,
    lowercase=True
)
print("CHRF: " + str(score))
print()

scores = rouge_score.compute(
    predictions=test_preds,
    references=test_refs,
    rouge_types=["rouge1", "rouge2", "rougeL"]
)

print("ROUGE:")
print(scores)
print()

score = bleu_score.compute(
    predictions=test_preds,
    references=test_refs
)
print("BLEU: " + str(score))
print()

In [ ]:
# CHRF example
score = chrf.compute(
    predictions=['In comparison, men are more than twice as likely to own a home alone or jointly (40%).'],
    references=['In comparison, men are more than twice as likely to own a home alone or jointly (20%).'],
    lowercase=True
)
print("CHRF: " + str(score))
print()

CHRF: {'score': 95.00381643651863, 'char_order': 6, 'word_order': 0, 'beta': 2}



In [ ]:
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.9 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("roberta-base", padding="max_length", truncation=True)
factkb = AutoModelForSequenceClassification.from_pretrained("bunsenfeng/FactKB", num_labels = 2)

def calc_fact(row):
  input = [[row['preds'], row['linearized_input']]]

  tokens = tokenizer(input, return_tensors="pt", padding="max_length", truncation=True)
  result = torch.softmax(factkb(**tokens).logits, dim = 1)

  return float(result[0][1])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

# Calculate and store the results in a list
results = []

for index, row in df.iterrows():
    result = calc_fact(row)
    results.append(result)

# Calculate the average of the results
average = sum(results) / len(results)

print(f"Average of calculated values: {average}")

Average of calculated values: 0.24333834883291275
